# [STARTER] Exercise - Build a Web-Aware Agent with Search and Knowledge Comparison

In this exercise, you'll build an agent that can search the web for current information and compare
it with its internal knowledge. This demonstrates how to enhance an LLM's capabilities with real-time
web data and how to critically analyze differences between sources.


## Challenge

Your task is to create an agent that can:

- Implement web search functionality using Tavily API
- Parse and process search results effectively
- Handle different types of queries (news, facts, events)
- Extract relevant information from search results

## Setup
First, let's import the necessary libraries:

In [21]:
import os
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [22]:
load_dotenv()

True

## Play with Tavily

In [23]:
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

In [24]:
# [TODO] Define a query and run
query = "how is Accenture doing in AI space?"
result = client.search(query)

In [25]:
result

{'query': 'how is Accenture doing in AI space?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.cxtoday.com/contact-center/accenture-lays-off-11000-staff-as-part-of-ai-reskilling-strategy/',
   'title': 'Accenture Lays Off 11000 Staff as Part of AI Reskilling Strategy',
   'content': '# Accenture Lays Off 11,000 Staff as Part of AI Reskilling Strategy However, despite the restructuring, plans for the new fiscal year expect the employee headcount to grow, particularly in areas such as AI and digital services. In the report, Accenture reveals its strategic realignment plan to increase its growth, upskilling and efficiency, driven by a demand for AI-related services, as well as an increase in employment. ## Accenture is All In on AI This restructuring is only the latest in Accenture’s plans to target the AI space. Despite the market’s less than enthusiastic response to Accenture’s layoffs, it will be interesting to see whether the company’s

## Define Web Search tool

In [43]:
# [TODO] Define the search tool
@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search
    """

    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)

    print(f"\nsearch_depth:  {search_depth}")
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )

    # Format the result
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "result": search_result.get("result", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }

    return formatted_results

In [44]:
tools = [web_search]

## Define Agents

The first agent should not use tools, just its own knowledge. The second one should have web search tool enabled.

In [45]:
# [TODO] Agent with no tools
simple_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=("you are a helpful assistant"),
)

In [46]:
# [TODO] Agent with web search tool
web_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
        "You are a web-aware assistant that can search for update information "
        "For each query, you will search the web for current information using " 
        "Tavily's AI-optimized search and provide a comprehensive answer\n"
        "Always cite your sources and explain any discrepancies found.\n"
        "Be particularly attentive to dates and time-sensitive information."
    ),
    tools=tools
)

In [47]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agents

Run the Agents and compare them. The following queries are just for reference. Change them as you want.

**Simple Agent**

**Note**: This example relies on the date being recent enough that the answer will not be in the model's training data. Try with other current events/dates if needed to get similar results.

In [48]:
run1 = simple_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = you are a helpful assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information about events or awards that occurred after October 2023. You may want to check a reliable news source or the official Oscars website for the most current information on the 2025 Oscars., tool_calls = None)


In [ ]:
print(run1.get_final_state()["messages"][-1].content)

In [ ]:
run2 = simple_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

In [ ]:
print(run2.get_final_state()["messages"][-1].content)

**Web Agent**

In [49]:
run1 = web_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor

search_depth:  advanced
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a web-aware assistant that can search for update information For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_5U4QLJ5zXKArxn5zr95dttIC', function=Function(arguments='{"query":"2025 Oscar winner for International Feature Film"}', name='

In [50]:
print(run1.get_final_state()["messages"][-1].content)

The winner of the 2025 Oscar for International Feature Film is "I'm Still Here," a film from Brazil. It was awarded at the 97th Academy Awards.


In [51]:
run2 = web_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor

search_depth:  advanced
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a web-aware assistant that can search for update information For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_5U4QLJ5zXKArxn5zr95dttIC', function=Function(arguments='{"query":"2025 Oscar winner for International Feature Film"}', name='

In [52]:
print(run2.get_final_state()["messages"][-1].content)

As of 2023, significant developments in AI technology include:

1. **Generative Models**: The launch of advanced generative models, such as Google's Gemini, which aim to enhance the capabilities and applications of AI in various fields.

2. **Multimodal Capabilities**: There has been a focus on developing AI systems that can understand and process multiple types of data simultaneously, such as text, image, and audio.

3. **Regulatory Frameworks**: The European Union introduced the AI Act, establishing guidelines and regulations to ensure responsible AI development and deployment.

These advancements highlight the rapid growth and evolving landscape of AI technology in 2023.


## Advanced

You can modify `agents.py` to include: 
- a comparison field in the state schema
- a web search step
- a comparison step in the workflow